In [1]:
import torch
import requests
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
'''Get Data from DB '''
def get_data_from_db():
    host = 'https://search-solytics-tzxvnvrvkscgklaedz5lr3iqxu.ap-south-1.es.amazonaws.com/news_test_list/_search'
    json_body = '''{
        "query": {
                "bool": {
                    "must_not": {
                        "exists":{
                            "field":"sentiment_ML"
                            }
                        }
                    }
                }
    }'''
    headers = {
        'Content-Type': 'application/json',
    }
    params = {
        'size':1000
    }
    resp = requests.get(host,params=params, headers=headers, data=json_body)
    resp_text = json.loads(resp.text)
    document_list = []

    # print(resp)
    for data in resp_text['hits']['hits']:
        content_list = {}
        content_list["id"] = data["_id"]
        content_list["content"] = data["_source"]["Content"]
        document_list.append(content_list)
    return document_list


In [3]:
device = torch.device('cuda') 

In [4]:
data = get_data_from_db()
df = pd.DataFrame(data)

In [5]:
from tqdm import tqdm

In [6]:
import time
import gc
start = time.time()
result=[]
'''GPU model'''
model = T5ForConditionalGeneration.from_pretrained('t5-small').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-small')
text_len = []
cnt=0
for text in tqdm(df['content']):
    try:
        '''If artice could not be possible to evaluate through GPU'''
        if(len(text.split())>2000):
            text = text.split()
            print(len(text))
            text = text[:2000]
            text = ' '.join(text)

        preprocess_text = text.strip().replace("\n","")
        t5_prepared_Text = "summarize: "+preprocess_text
        tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
        summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=100,
                                        early_stopping=True)
        output = tokenizer.decode(summary_ids[0])
        print(output)
        result.append(output)
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(e)
        print(len(text.split()))
        gc.collect()
        torch.cuda.empty_cache()

gc.collect()
torch.cuda.empty_cache()

del model
torch.cuda.empty_cache()

del tokenizer
torch.cuda.empty_cache()

  0%|          | 0/574 [00:00<?, ?it/s]

  0%|          | 1/574 [00:03<30:49,  3.23s/it]

the ROG Strix Hero II isn’t the best gaming notebook that money can buy. however, there aren't many that can outperform it on a strictly price-for-performance basis. if you’re looking for something like Call of Duty, Counter-Strike or Overwatch, the Hero is the option you want.


  0%|          | 2/574 [00:04<25:48,  2.71s/it]

archbishop Ezekiel Kondo Kumir Kuku has been installed as the first Archbisbisp of the newest 39th province of anglican Communion in the Sudan. by the Archhop of Canterbury, by... at the All... the all....... he is the only archi... to be installed in a new state - the new archb... is 'the first archa... by anarchia! 


...

  4%|▍         | 23/574 [00:41<14:35,  1.59s/it]

sign up for our newsletter and discover travel experiences you’ll really want to try.By signing up, you agree to receive promotional emails. you can unsubscribe at any time.


  4%|▍         | 24/574 [00:43<16:43,  1.82s/it]

la Polic ⁇ a Nacional ha detenido en Madrid al narco colombiano Germán Gonzalo Sánchez Rey. la operación se ha coordinado de forma conjunta con la oficina central cianional de INTERPOL in Madrid. l'argumento está relacionado con el tráfico de coca ⁇ 


KeyboardInterrupt: 

In [10]:
result[5]

'Iran has sought "permanent authorisation" from India for its two local insurance companies. the proposal was put forth by a Managing Director at the India office of Iran\'s state-run national Iranian oil company last month. previously, India struggled to get tankers and insurance for transporting oil from Iran after the US and the European Union imposed sanctions between July 2012 and January 2014 on the Islamic Republic.'

In [11]:
df['content'][5]

'Iran has sought "permanent authorisation" from India for its two local insurance companies to enable Iranian ships with Iranian insurance cover to call at Indian ports to help maintain oil supplies from the Persian Gulf nation when new sanctions imposed by the US take effect in November.\nThe proposal was put forth by M Sarmadi, Managing Director at the India office of Iran’s state-run National Iranian Oil Company, during a meeting with the top officials of the Shipping Ministry last month, reports the Hindu Business Line citing a document related to this issue.\nPreviously, India struggled to get tankers and insurance for transporting oil from Iran after the US and the European Union imposed sanctions between July 2012 and January 2014 on the Islamic Republic, forcing western insurers to halt insuring ships hauling crude from Iran. Following the sanctions, London-based International Group of Protection and Indemnity Clubs stopped providing third party liability cover to ships hauling

In [ ]:
text = '''Prime Minister Narendra Modi’s first stop in Uttar Pradesh’s Ayodhya on August 5 will be a shrine dedicated to Lord Hanuman, where he will offer prayers ahead of the grand foundation laying ceremony for the construction of the Ram temple.

Mahant Raju Das, the head priest of Hanumangarhi, said they have been allocated seven minutes for Prime Minister’s rituals at the temple, according to news agency ANI.

Vedic priests will escort the Prime Minister while reciting mantras to the Hanumangarhi temple.'''

In [ ]:
preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
summary_ids = model.generate(tokenized_text,
                                num_beams=4,
                                no_repeat_ngram_size=2,
                                min_length=30,
                                max_length=100,
                                early_stopping=True)
output = tokenizer.decode(summary_ids[0])
print(output)